In [24]:
import random
import networkx  
import pgmpy.models as pgmm  
import pgmpy.factors.discrete as pgmf              
import pgmpy.inference as pgmi
import random

In [25]:
#Entrada de datos del buscaminas
filas = 6
columnas = 6
minas = 6

In [89]:
# Ahora, debemos de crear el tablero del juego, el cual dependerá del numero de filas, columnas y minas
def tablero_buscaminas(filas, columnas, minas):
    # Antes de nada, se debe de comprobar que el número de minas existente en el tablero sea menor o igual al número de casillas
    if(minas > filas * columnas):
        raise Exception ("El número de minas debe ser menor o igual a" + filas*columnas)
        
    # A continuación se pasa a construir el tablero
    tableroBuscaminas = [[0 for x in range(filas)] for y in range(columnas)]
    
    # Vamos a pintar las minas. Recorremos el número de minas que queremos pintar y en el tablero mostramos "X" que indica
    # que en esa casilla existe una mina.
    for i in range(minas):
        tableroBuscaminas[random.randint(0,filas-1)][random.randint(0,columnas-1)] = "X"
        
    # Ahora pasamos a poner los índices de las casillas colindantes con minas. El número que aparece en la casilla va en un
    # rango de 1 a 8 (arriba, abajo, izquierda, derecha y las 4 diagonales).
    minas_colindantes = 0
    for fila in range(filas):
        for columna in range(columnas):
            if(fila > 0 and tableroBuscaminas[fila-1][columna] == "X"): # La mina se encuentra arriba
                minas_colindantes += 1
            if(fila +1 < len(tableroBuscaminas) and tableroBuscaminas[fila+1][columna] == "X"): # La mina se encuentra abajo
                minas_colindantes +=1
            if(columna+1 < len(tableroBuscaminas[0]) and tableroBuscaminas[fila][columna+1] == "X"):
                # La mina se encuentra a la derecha
                minas_colindantes +=1
            if(columna > 0 and tableroBuscaminas[fila][columna-1] == "X"): # La mina se encuentra a la izquierda
                minas_colindantes +=1
            if(fila > 0 and columna+1 < len(tableroBuscaminas[0]) and tableroBuscaminas[fila-1][columna+1] == "X"):
                # La mina se encuentra en la diagonal superior derecha
                minas_colindantes +=1
            if(fila+1 < len(tableroBuscaminas) and columna+1 < len(tableroBuscaminas[0]) and 
                   tableroBuscaminas[fila+1][columna+1] == "X"): 
                # La mina se encuentra en la diagonal inferior derecha
                minas_colindantes +=1
            if(fila > 0 and columna > 0 and tableroBuscaminas[fila-1][columna-1] == "X"):
                # La mina se encuentra en la diagonal superior izquierda
                minas_colindantes +=1
            if(fila+1 < len(tableroBuscaminas) and columna > 0 and tableroBuscaminas[fila+1][columna-1] == "X"): 
                # La mina se encuentra en la diagonal inferior izquierda
                minas_colindantes +=1
            # Pasamos a guardar el índice de minas colindantes en la casilla del tablero donde no hay ninguna mina.
            if(minas_colindantes > 0 and tableroBuscaminas[fila][columna] != "X" ):
                tableroBuscaminas[fila][columna] = str(minas_colindantes)
            minas_colindantes=0
            
    return tableroBuscaminas
    
    
    
    

In [90]:
# Prueba de creación del tablero con los parámetros indicados
tableroBuscaminas = tablero_buscaminas(filas,columnas,minas)
tableroBuscaminas

[['X', '2', 'X', '1', '1', 'X'],
 ['1', '2', '1', '1', '1', '1'],
 [0, 0, 0, '1', '1', '1'],
 [0, 0, '1', '2', 'X', '2'],
 [0, 0, '1', 'X', '3', 'X'],
 [0, 0, '1', '1', '2', '1']]

In [91]:
# Pasamos a ocultar las casillas del tablero, se determina que dichas casillas ocultas se muestran con el símbolo "O"
def ocultar_casillas_tablero(filas,columnas):
    tablero_buscaminas_oculto = [['O' for columna in range(columnas)] for fila in range(filas)]
        
    return tablero_buscaminas_oculto

In [92]:
# Prueba de ocultación del tablero
tablero_buscaminas_oculto = ocultar_casillas_tablero(filas,columnas)
tablero_buscaminas_oculto

[['O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O']]

In [99]:
# Vamos ahora a destapar las casillas del tablero. Al clicar sobre una casilla que no contiene una mina, se van a destapar un
# conjunto colindante de casillas que no tienen minas ni están proximas con otras, es decir, con índice = 0. Como máximo,
# el límite del destapado va a ser si la casilla que se destapa tiene al menos un vecino con mina.
def destapar_casillas_tablero(fila,columna):
    if(tableroBuscaminas[fila][columna] == "Mina"):
        raise Exception("Has encontrado una mina. Vuelve a intentarlo")
    else:
        if(tableroBuscaminas[fila][columna] == 0): #Comprobamos si la casilla que destapamos tiene 0 minas colindantes
            # si se da el caso, destapamos las colindantes a esa casilla
            if(tableroBuscaminas[fila-1][columna] == 0):
                destapar_casillas_tablero(fila-1,columna)
            if(tableroBuscaminas[fila+1][columna] == 0):
                destapar_casillas_tablero(fila+1,columna)
            if(tableroBuscaminas[fila][columna+1] == 0):
                destapar_casillas_tablero(fila,columna+1)
            if(tableroBuscaminas[fila][columna-1] == 0):
                destapar_casillas_tablero(fila,columna-1)
            if(tableroBuscaminas[fila-1][columna+1] == 0):
                destapar_casillas_tablero(fila-1,columna+1)
            if(tableroBuscaminas[fila+1][columna+1] == 0):
                destapar_casillas_tablero(fila+1,columna+1)
            if(tableroBuscaminas[fila-1][columna-1] == 0):
                destapar_casillas_tablero(fila-1,columna-1)
            if(tableroBuscaminas[fila+1][columna-1] == 0):
                destapar_casillas_tablero(fila+1,columna-1)
    
    return tablero_buscaminas_oculto        
            
    
            

In [103]:
# Vamos a probar ahora el destapado de una casilla
destapar_casillas_tablero(0,0)

[['O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O']]